<a href="https://colab.research.google.com/github/aioont/KH012-FinWise/blob/main/FinWise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install keras

In [3]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#for debug
from IPython.display import clear_output
from google.colab import files

Data Preprocessing

In [ ]:
#read file
df = pd.read_csv("/content/nsestockhistoricalratios.csv")

#Median Imputation
def fill_with_median(column):
  try:
    column_median = column.median(skipna=True)
  except (ValueError, TypeError):
    column_median = 0.0
  return column.fillna(column_median)

for col in df.columns:
    df[col] = fill_with_median(df[col])

#split dataframe on companies
stock = {}
for index, row in df.iterrows():
    key = row['Stock']
    if key not in stock:
        stock[key] = pd.DataFrame(columns=df.columns)
        continue
    stock[key] = pd.concat([stock[key], row.to_frame().T], ignore_index=True)

#reshaping dataframe
drop = []
for key in stock:
  stock[key].drop(['Stock'], axis=1, inplace=True)
  company = pd.pivot_table(stock[key], index="Year")

  #min-max scaling
  for col in company.columns:
    normalized_values = (company[col] - np.min(company[col])) / (np.max(company[col]) - np.min(company[col]))
    company.loc[:, col] = normalized_values

  stock[key] = company
  num_rows = stock[key].shape[0]

  if num_rows:
    file_name = f"dataset/forheatmap_{key}.csv"
    stock[key].to_csv(file_name, index=True)
  else:
    drop.append(key)

for key in drop:
  stock.pop(key)

In [ ]:
##debug##

# df = pd.read_csv("/content/nsestockhistoricalratios.csv", na_values=['', ',,', np.NaN, 'NULL'])

# stock = {}
# for index, row in df.iterrows():
#     key = row['Stock']
#     if key not in stock:
#         stock[key] = pd.DataFrame(columns=df.columns)
#         continue
#     stock[key] = pd.concat([stock[key], row.to_frame().T], ignore_index=True)

i = 1
for key in stock:
  stock[key].dropna(axis=0)
  dataframe_size = stock[key].shape   # Get the size of the DataFrame
  num_rows, num_columns = dataframe_size   # Extract the number of rows and columns
  num_na = stock[key].isna().sum().sum()
  if num_rows and num_columns == 98 and num_na == 0:
    print(f"{i}) {key}: rows={num_rows}; columns={num_columns}; NaN={num_na}")
    i+=1

  # input("Enter to continue")
  # clear_output(wait=True)

!zip -r /content/file.zip /content/dataset
files.download("/content/file.zip")


In [ ]:
for key in stock:
  file_name = f"/content/dataset/forheatmap_{key}.csv"
  na = [-99, 99, 'NULL', '']
  stock[key] = pd.read_csv(file_name, na_values=na)

  print(stock[key].isna().sum().sum())

In [ ]:
#Creating Heatmaps for each company
for key in list(stock.keys()):
  stock[key].drop(['Stock'], axis=1, inplace=True)
  company = pd.pivot_table(stock[key], index="Year")

  #min-max scaling
  for col in company.columns:
    company[col] = (company[col] - np.min(company[col])) / (np.max(company[col]) - np.min(company[col]))

  sns.heatmap(company, cmap="YlGnBu", fmt="0.0f")
  # filename = f"heatmap_{key}.png"
  # plt.savefig(filename)

Model Architecture

In [ ]:
ClassifyStock = Sequential([
    Conv2D(64, (58,5), strides=(5,5), activation='relu', input_size=(640,480,3)),
    MaxPool2D((5,5)),

    Conv2D(64, (58,5), strides=(5,5), activation='relu', input_size=(640,480,3)),
    MaxPool2D((5,5)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

Model Compile

In [ ]:
ClassifyStock.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])